# PyTorch Fashion MNIST example

"Hello world" type example, based on https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torchvision
from dotmap import DotMap
from magic_timer import MagicTimer
from torch.utils.data import DataLoader
from tqdm import tqdm

In [7]:
# print versions of imported libraries if can
import sys

for module_name in sorted(set(sys.modules) & set(globals())):
    if version := getattr(sys.modules[module_name], "__version__", None):
        print(module_name, version)

torch 1.12.1
torchvision 0.13.1
tqdm 4.64.1


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device = }")
cf = DotMap(
    {
        "learning_rate": 1e-3,
        "batch_size": 64,
        "num_epochs": 10,
    }
)

device = 'cuda'


In [4]:
training_data = torchvision.datasets.FashionMNIST(
    root="data", train=True, download=True, transform=torchvision.transforms.ToTensor()
)
test_data = torchvision.datasets.FashionMNIST(
    root="data", train=False, download=True, transform=torchvision.transforms.ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=cf.batch_size)
test_dataloader = DataLoader(test_data, batch_size=cf.batch_size)

In [5]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_loop(dataloader, model, loss_fn, optimizer, epoch):
    pbar = tqdm(dataloader, desc=f"Epoch {epoch}")
    for batch, (X, y) in enumerate(pbar):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test set: Accuracy = {(100*correct):.1f}%, Mean loss = {test_loss:.4f}")

In [6]:
# Train for `cf.num_epochs` epochs
timer = MagicTimer()
model = Model().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=cf.learning_rate)
for i in range(1, cf.num_epochs + 1):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch=i)
    test_loop(test_dataloader, model, loss_fn)
print(f"Trained model in {timer}.")

Epoch 1: 100%|██████████| 938/938 [00:08<00:00, 106.90it/s, loss=2.1174]


Test set: Accuracy = 43.9%, Mean loss = 2.1298


Epoch 2: 100%|██████████| 938/938 [00:08<00:00, 117.07it/s, loss=1.8377]


Test set: Accuracy = 59.7%, Mean loss = 1.8251


Epoch 3: 100%|██████████| 938/938 [00:07<00:00, 119.71it/s, loss=1.4907]


...

Test set: Accuracy = 69.2%, Mean loss = 0.8381


Epoch 9: 100%|██████████| 938/938 [00:07<00:00, 119.75it/s, loss=0.8842]


Test set: Accuracy = 70.7%, Mean loss = 0.8006


Epoch 10: 100%|██████████| 938/938 [00:07<00:00, 121.91it/s, loss=0.8572]


Test set: Accuracy = 72.0%, Mean loss = 0.7704
Trained model in 1.5 minutes.
